In [4]:
from PIL import Image
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/new_image/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        imageData = Image.open(f"""../dataset_dir/new_image/{dir}/{file}""").convert("L") # 이미지 tensor 데이터 출력
        imageData = imageData.resize((50, 50)) # 이미지 업스케일링

        imageData = np.array(imageData)
        all_x_array.append(imageData.tolist())

        y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
        y_array = list(y_array.values())
        all_y_array.append(y_array)


In [5]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [6]:
print(x_array.shape)
print(y_array.shape)

(794, 50, 50)
(794, 13)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, shuffle=False, random_state=1004
)

In [8]:
X_train = X_train.astype("float32") / 255
Y_train = Y_train.astype("float32") / 255

X_test = X_test.astype("float32") / 255
Y_test = Y_test.astype("float32") / 255

In [9]:
print(X_train.shape)
print(X_test.shape)

print(Y_train.shape)
print(Y_test.shape)

(635, 50, 50)
(159, 50, 50)
(635, 13)
(159, 13)


In [10]:
u = np.unique(Y_train, axis=0)
len(u)

11

In [11]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(50, 50, 1)),
    keras.layers.Conv2D(32, (4, 4), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(13, activation="softmax"),
])

2024-02-26 17:07:57.367515: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 17:07:57.419059: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 17:07:57.419110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 17:07:57.420302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 17:07:57.426191: I tensorflow/core/platform/cpu_feature_guar

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 47, 47, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [15]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.001), metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/15
5/5 [==============================] - 1s 86ms/step - loss: 18131.7129 - accuracy: 0.0693 - val_loss: 2567.9487 - val_accuracy: 0.0000e+00
Epoch 2/15
5/5 [==============================] - 0s 72ms/step - loss: 17834.6484 - accuracy: 0.0724 - val_loss: 2638.2429 - val_accuracy: 0.0000e+00
Epoch 3/15
5/5 [==============================] - 0s 79ms/step - loss: 17761.8887 - accuracy: 0.0614 - val_loss: 2379.5215 - val_accuracy: 0.0000e+00
Epoch 4/15
5/5 [==============================] - 0s 83ms/step - loss: 18090.5645 - accuracy: 0.0709 - val_loss: 2186.7141 - val_accuracy: 0.0000e+00
Epoch 5/15
5/5 [==============================] - 0s 83ms/step - loss: 18007.2930 - accuracy: 0.0819 - val_loss: 2168.2664 - val_accuracy: 0.0000e+00
Epoch 6/15
5/5 [==============================] - 0s 84ms/step - loss: 18667.5645 - accuracy: 0.0693 - val_loss: 1804.4877 - val_accuracy: 0.0000e+00
Epoch 7/15
5/5 [==============================] - 0s 68ms/step - loss: 19573.8828 - accuracy: 0.0740